<a href="https://colab.research.google.com/github/dosanchez/data-science-learning/blob/main/exercises_chpt_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>